In [ ]:
import cv2
import cv2.face
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
def detect_face(frame):
    detector =cv2.CascadeClassifier('xml/haarcascade_frontalface_alt.xml')
    faces=detector.detectMultiScale(frame,scaleFactor=1.3,minNeighbors=3)
    return faces

def cut_faces(image,faces_coord):
    faces=[]
    for (x,y,w,h) in faces_coord:
        faces.append(image[y:y+h,x:x+w])
    return faces
        
def normalize_intensity(images):
    images_norm=[]
    for image in images:
        is_color=len(image.shape)==3
        if is_color:
            image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        images_norm.append(cv2.equalizeHist(image))            #increase brightness of picture
    return images_norm

def resize(images,size=(47,62)):
    image_resize=[]
    for image in images:
        if image.shape<size:
            img_size=cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size=cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)
    return image_resize

def normalize_faces(frame,faces_coord):
    faces=cut_faces(frame,faces_coord)
    faces=normalize_intensity(faces)
    faces=resize(faces)
    return faces

def plot_show(image,title=""):
    if len(image.shape)==3:
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()
    
def draw_rectangle(image,coords):
    for(x,y,w,h) in coords:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)

In [ ]:
def collect_dataset():
    images=[]
    labels=[]
    labels_dic={}
    trainingData=[gender for gender in os.listdir("trainingData/")]
    for i,gender in enumerate(trainingData):
        labels_dic[i]=gender
        for image in os.listdir("trainingData/"+gender):
            if image.endswith('.jpg'):
                images.append(cv2.imread("trainingData/"+gender+'/'+image,0))
                labels.append(i)
    return (images,np.array(labels),labels_dic)
        

In [ ]:
images,labels,labels_dic=collect_dataset()
labels_dic
#labels

In [ ]:
x_train=np.asarray(images)

In [ ]:
train=x_train.reshape(len(x_train),-1)
x_train.shape

In [ ]:
from sklearn.decomposition import PCA
pca1 = PCA()
pca1.fit(train)
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
pca1 = PCA(n_components=.99)
new_train=pca1.fit_transform(train)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:
param_grid = {'C':[.0001,.001,.01,.1,1,10]}

In [ ]:
kf=KFold(n_splits=5,shuffle=True)

In [ ]:
gs_svc = GridSearchCV(SVC(kernel='linear',probability=True),param_grid=param_grid,cv=kf,scoring='accuracy')

In [ ]:
gs_svc.fit(new_train,labels)

In [ ]:
gs_svc.best_score_

In [ ]:
gs_svc.best_params_

In [ ]:
svc1 = gs_svc.best_estimator_

In [ ]:
import pickle
filename = 'svc_gender.pkl'
f=open(filename, 'wb')
pickle.dump(svc1,f)
f.close()


In [ ]:
import pickle

filename = 'svc_gender.pkl'
svc1 = pickle.load(open(filename, 'rb'))

In [ ]:
#face.ndim

In [ ]:
from IPython.display import clear_output
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    
    faces_coord = detect_face(frame) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            test = pca1.transform(face.reshape(-1,1))    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print (confidence)
            print (prob)
           
            
            
            pred = svc1.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            #pred = labels_dic[pred[0]].capitalize()
            #threshold = .50
            
            if prob[0][1]>.75:
                
                cv2.putText(frame, 'MALE',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][0]>.80:
                cv2.putText(frame,'FEMALE',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
                
            #cv2.putText(frame,name,(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
            
               
           #cv2.putText(frame,'Unknown',(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,
                cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        